In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import (
    StructType,
    StructField,
    StringType,
    IntegerType,
    FloatType,
)
import findspark
import numpy as np


In [2]:
findspark.init()


In [3]:
# Creating a spark context class
sc = SparkContext()

# Creating a spark session
"""
spark = (
    SparkSession.builder.appName("Semana4")
    .config("spark.some.config.option", "some-value")
    .getOrCreate()
)
"""
spark = SparkSession.builder.appName("Semana4").master("local[*]")


In [92]:
path_airports = "../Data/airports_qa.parquet"

df_airports_qa = (
    spark.getOrCreate()
    .read.format("parquet")
    .option("header", "true")
    .load(path_airports)
)

path_planes = "../Data/planes_qa.parquet"

df_planes_qa = (
    spark.getOrCreate()
    .read.format("parquet")
    .option("header", "true")
    .load(path_planes)
)


path_flights = "../Data/flights_qa.parquet"

df_flights_qa = (
    spark.getOrCreate()
    .read.format("parquet")
    .option("header", "true")
    .load(path_flights)
)
path_airports = "../Data/airports_proc.parquet"
df_airports_proc = (
    spark.getOrCreate()
    .read.format("parquet")
    .option("header", "true")
    .load(path_airports)
)

path_planes = "../Data/planes_proc.parquet"

df_planes_proc = (
    spark.getOrCreate()
    .read.format("parquet")
    .option("header", "true")
    .load(path_planes)
)


path_flights = "../Data/flights_proc.parquet"

df_flights_proc = (
    spark.getOrCreate()
    .read.format("parquet")
    .option("header", "true")
    .load(path_flights)
)


# Pergunta qualidade

Pergunta 1

In [6]:
df = df_flights_qa.join(
    df_airports_qa, df_flights_qa.origin == df_airports_qa.faa, "left"
)
df_airports_qa_1 = (
    df_airports_qa.withColumnRenamed("faa", "faa_dest")
    .withColumnRenamed("qa_faa", "qa_faa_dest")
    .withColumnRenamed("qa_name", "qa_name_dest")
    .withColumnRenamed("qa_lat", "qa_lat_dest")
    .withColumnRenamed("qa_lon", "qa_lon_dest")
    .withColumnRenamed("qa_tz", "qa_tz_dest")
    .withColumnRenamed("qa_dst", "qa_dst_dest")
    .withColumnRenamed("qa_alt", "qa_alt_dest")
)

df = df.join(df_airports_qa_1, df.dest == df_airports_qa_1.faa_dest, "left")
df_planes_qa = df_planes_qa.withColumnRenamed("qa_tailnum", "qa_tailnum_plane")
df = df.join(df_planes_qa, df.tailnum == df_planes_qa.tailnum, "left")

df.printSchema()


root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- dep_time: string (nullable = true)
 |-- dep_delay: double (nullable = true)
 |-- arr_time: string (nullable = true)
 |-- arr_delay: double (nullable = true)
 |-- carrier: string (nullable = true)
 |-- tailnum: string (nullable = true)
 |-- flight: string (nullable = true)
 |-- origin: string (nullable = true)
 |-- dest: string (nullable = true)
 |-- air_time: double (nullable = true)
 |-- distance: integer (nullable = true)
 |-- hour: double (nullable = true)
 |-- minute: double (nullable = true)
 |-- qa_year_month_day: integer (nullable = true)
 |-- qa_hour_minute: string (nullable = true)
 |-- qa_dep_arr_time: string (nullable = true)
 |-- qa_dep_arr_delay: string (nullable = true)
 |-- qa_carrier: integer (nullable = true)
 |-- qa_tailnum: string (nullable = true)
 |-- qa_flight: string (nullable = true)
 |-- qa_origin_dest: integer (nullable = true)
 |-- qa_air_t

Pergunta 2

In [8]:
df_qa = df.select(df.colRegex("`^qa_.*`"))

for c in df_qa.columns:
    df_qa.filter(F.col(c).isNotNull()).groupBy(
        F.substring(c, 1, 1).alias(c)
    ).count().show()


+-----------------+-----+
|qa_year_month_day|count|
+-----------------+-----+
+-----------------+-----+

+--------------+-----+
|qa_hour_minute|count|
+--------------+-----+
|             M|   48|
+--------------+-----+

+---------------+-----+
|qa_dep_arr_time|count|
+---------------+-----+
|              F|  241|
|              M|   55|
+---------------+-----+

+----------------+-----+
|qa_dep_arr_delay|count|
+----------------+-----+
|               M|   75|
+----------------+-----+

+----------+-----+
|qa_carrier|count|
+----------+-----+
+----------+-----+

+----------+-----+
|qa_tailnum|count|
+----------+-----+
|         F|  989|
|         M|   14|
+----------+-----+

+---------+-----+
|qa_flight|count|
+---------+-----+
|        F| 6158|
+---------+-----+

+--------------+-----+
|qa_origin_dest|count|
+--------------+-----+
+--------------+-----+

+-----------+-----+
|qa_air_time|count|
+-----------+-----+
|          M|   75|
+-----------+-----+

+-----------+-----+
|qa_distanc

Pergunta 3

In [72]:
maxi = list()
col = list()
for c in df_qa.columns:
    temp = (
        df_qa.filter(F.col(c) == "M")
        .groupBy(F.substring(c, 1, 1).alias(c))
        .count()
    )
    if len(temp.filter(F.col("count") > 0).collect()):
        if temp.filter(F.col("count") > 0).collect()[0][1]:
            maxi.append(temp.filter(F.col("count") > 0).collect()[0][1])
            col.append(c)
maxi_np = np.array(maxi)

print(f"Variavel {col[np.argmax(maxi_np)]} com {np.max(maxi_np)}.")


Variavel qa_speed com 9443.


Pergunta 4

In [75]:
for c in df_qa.columns:
    df_qa.filter(F.col(c) == "F").groupBy(
        F.substring(c, 1, 1).alias(c)
    ).count().show()


+-----------------+-----+
|qa_year_month_day|count|
+-----------------+-----+
+-----------------+-----+

+--------------+-----+
|qa_hour_minute|count|
+--------------+-----+
+--------------+-----+

+---------------+-----+
|qa_dep_arr_time|count|
+---------------+-----+
+---------------+-----+

+----------------+-----+
|qa_dep_arr_delay|count|
+----------------+-----+
+----------------+-----+

+----------+-----+
|qa_carrier|count|
+----------+-----+
+----------+-----+

+----------+-----+
|qa_tailnum|count|
+----------+-----+
|         F|  987|
+----------+-----+

+---------+-----+
|qa_flight|count|
+---------+-----+
|        F| 6158|
+---------+-----+

+--------------+-----+
|qa_origin_dest|count|
+--------------+-----+
+--------------+-----+

+-----------+-----+
|qa_air_time|count|
+-----------+-----+
+-----------+-----+

+-----------+-----+
|qa_distance|count|
+-----------+-----+
+-----------+-----+

+-------------------+-----+
|qa_distance_airtime|count|
+-------------------+-----+
+

Pergunta 5

In [76]:
for c in df_qa.columns:
    df_qa.filter(F.col(c) == "I").groupBy(
        F.substring(c, 1, 1).alias(c)
    ).count().show()


+-----------------+-----+
|qa_year_month_day|count|
+-----------------+-----+
+-----------------+-----+

+--------------+-----+
|qa_hour_minute|count|
+--------------+-----+
+--------------+-----+

+---------------+-----+
|qa_dep_arr_time|count|
+---------------+-----+
+---------------+-----+

+----------------+-----+
|qa_dep_arr_delay|count|
+----------------+-----+
+----------------+-----+

+----------+-----+
|qa_carrier|count|
+----------+-----+
+----------+-----+

+----------+-----+
|qa_tailnum|count|
+----------+-----+
+----------+-----+

+---------+-----+
|qa_flight|count|
+---------+-----+
+---------+-----+

+--------------+-----+
|qa_origin_dest|count|
+--------------+-----+
+--------------+-----+

+-----------+-----+
|qa_air_time|count|
+-----------+-----+
+-----------+-----+

+-----------+-----+
|qa_distance|count|
+-----------+-----+
+-----------+-----+

+-------------------+-----+
|qa_distance_airtime|count|
+-------------------+-----+
+-------------------+-----+

+------+-

Pergunta negocio

Pergunta 1

In [99]:
df = 0


In [105]:
df_1 = df_airports_proc
df_2 = df_airports_proc


print("Actual columns:", df_1.columns)


df_1 = (
    df_1.withColumnRenamed("faa", "faa_O")
    .withColumnRenamed("name", "name_O")
    .withColumnRenamed("lat", "lat_O")
    .withColumnRenamed("lon", "lon_O")
    .withColumnRenamed("alt", "alt_O")
    .withColumnRenamed("tz", "tz_O")
    .withColumnRenamed("dst", "dst_O")
    .withColumnRenamed("region", "region_O")
    .withColumnRenamed("type", "type_O")
    .withColumnRenamed("military", "military_O")
    .withColumnRenamed("administration", "administration_O")
)
print("modified columns:", df_1.columns)

df_1.toPandas()


print("Actual columns:", df_2.columns)


df_2 = (
    df_2.withColumnRenamed("faa", "faa_D")
    .withColumnRenamed("name", "name_D")
    .withColumnRenamed("lat", "lat_D")
    .withColumnRenamed("lon", "lon_D")
    .withColumnRenamed("alt", "alt_D")
    .withColumnRenamed("tz", "tz_D")
    .withColumnRenamed("dst", "dst_D")
    .withColumnRenamed("region", "region_D")
    .withColumnRenamed("type", "type_D")
    .withColumnRenamed("military", "military_D")
    .withColumnRenamed("administration", "administration_D")
)
print("modified columns:", df_2.columns)

df_2.toPandas()


print("Actual columns:", df_planes_proc.columns)


df_planes_proc = df_planes_proc.withColumnRenamed("tailnum", "tailnum_P")

print("modified columns:", df_planes_proc.columns)

df_planes_proc.toPandas()


df = df_flights_proc.join(df_1, (df_flights_proc.origin == df_1.faa_O), "left")


df = df.join(df_2, (df.dest == df_2.faa_D), "left")

df.printSchema()


Actual columns: ['faa', 'name', 'lat', 'lon', 'alt', 'tz', 'dst', 'region', 'type', 'military', 'administration']
modified columns: ['faa_O', 'name_O', 'lat_O', 'lon_O', 'alt_O', 'tz_O', 'dst_O', 'region_O', 'type_O', 'military_O', 'administration_O']
Actual columns: ['faa', 'name', 'lat', 'lon', 'alt', 'tz', 'dst', 'region', 'type', 'military', 'administration']
modified columns: ['faa_D', 'name_D', 'lat_D', 'lon_D', 'alt_D', 'tz_D', 'dst_D', 'region_D', 'type_D', 'military_D', 'administration_D']
Actual columns: ['tailnum_P', 'year', 'type', 'manufacturer', 'model', 'engines', 'seats', 'speed', 'engine', 'tailchar', 'age', 'engine_type']
modified columns: ['tailnum_P', 'year', 'type', 'manufacturer', 'model', 'engines', 'seats', 'speed', 'engine', 'tailchar', 'age', 'engine_type']
root
 |-- dep_time: string (nullable = true)
 |-- dep_delay: integer (nullable = true)
 |-- arr_time: string (nullable = true)
 |-- arr_delay: integer (nullable = true)
 |-- carrier: string (nullable = true

Pergunta 2

In [106]:
df.groupBy(F.col("region_D")).agg(
    F.countDistinct("faa_D").alias("aeroportos_por_regiao")
).toPandas()


,region_D,aeroportos_por_regiao
0,ALASKA,9
1,MAINLAND-EAST,24
2,MAINLAND-WEST,36


Pergunta 3

In [107]:
df.agg(
    F.max(F.col("alt_O") - F.col("alt_D")).alias("maior_dif_altitude")
).toPandas()


,maior_dif_altitude
0,429


Pergunta 4

In [110]:
df.select("dep_delay", "arr_delay").where("dep_delay > 0 OR arr_delay > 0").agg(
    F.ceil(F.avg(F.col("dep_delay") + F.col("arr_delay"))).alias("atraso_medio")
).toPandas()


,atraso_medio
0,35


Pergunta 5

In [111]:
df.where("dep_delay > 0 OR arr_delay > 0").groupBy("region_D").agg(
    F.ceil(F.avg(F.col("dep_delay") + F.col("arr_delay"))).alias(
        "atraso_medio_regiao (min)"
    )
).toPandas()


,region_D,atraso_medio_regiao (min)
0,ALASKA,28
1,MAINLAND-EAST,37
2,MAINLAND-WEST,36


Pergunta 6

In [112]:
df.where((F.col("dep_delay") > 0) | (F.col("arr_delay") > 0)).groupBy(
    F.year(F.col("dep_datetime")).alias("ano")
).agg(
    F.sum(F.col("dep_delay") + F.col("arr_delay")).alias(
        "atraso_acumulado_ano (min)"
    )
).toPandas()


,ano,atraso_acumulado_ano (min)
0,2014,166798


Pergunta 7

In [113]:
df.select(
    F.year("dep_datetime").alias("ano"), "dep_delay", "arr_delay", "region_D"
).where("dep_delay > 0 OR arr_delay > 0").groupBy("ano", "region_D").agg(
    F.sum(F.col("dep_delay") + F.col("arr_delay")).alias(
        "atraso_acumulado_ano (min)"
    )
).toPandas()


,ano,region_D,atraso_acumulado_ano (min)
0,2014,ALASKA,14281
1,2014,MAINLAND-EAST,40558
2,2014,MAINLAND-WEST,111959


Pergunta 8

In [114]:
df.select("air_time").agg(
    F.ceil(F.avg("air_time")).alias("tempo_voo_medio")
).toPandas()


,tempo_voo_medio
0,153


Pergunta 9

In [115]:
df.select("air_time", "region_D").groupBy("region_D").agg(
    F.ceil(F.avg("air_time")).alias("tempo_voo_medio_regiao")
).toPandas()


,region_D,tempo_voo_medio_regiao
0,ALASKA,228
1,MAINLAND-EAST,238
2,MAINLAND-WEST,116


Pergunta 10

In [116]:
df.select("air_time", "origin", "dest").groupBy("origin", "dest").agg(
    F.ceil(F.avg("air_time")).alias("tempo_voo_medio_rota")
).toPandas()


,origin,dest,tempo_voo_medio_rota
0,SEA,RNO,75
1,SEA,DTW,220
2,SEA,CLE,234
3,SEA,LAX,127
4,PDX,SEA,35
...,...,...,...
108,SEA,JFK,282
109,SEA,FAI,199
110,PDX,HOU,227
111,PDX,BUR,109


Pergunta 11

In [117]:
df.select(F.year("dep_datetime").alias("ano"), "air_time").groupBy("ano").agg(
    F.sum("air_time").alias("tempo_voo_acumulado_ano")
).toPandas()


,ano,tempo_voo_acumulado_ano
0,2013,30
1,2014,1528595


Pergunta 12

In [118]:
df.select(F.year("dep_datetime").alias("ano"), "air_time", "region_D").groupBy(
    "ano", "region_D"
).agg(F.sum(F.col("air_time")).alias("tempo_voo_acumulado_regiao")).toPandas()


,ano,region_D,tempo_voo_acumulado_regiao
0,2014,ALASKA,230602
1,2014,MAINLAND-EAST,508344
2,2014,MAINLAND-WEST,789649
3,2013,MAINLAND-WEST,30


Pergunta 13

In [120]:
df.select("distance").agg(
    F.ceil(F.avg(F.col("distance"))).alias("distancia_voo_medio")
).toPandas()


,distancia_voo_medio
0,1209


Pergunta 14

In [121]:
df.select("distance", "region_D").groupBy("region_D").agg(
    F.ceil(F.avg(F.col("distance"))).alias("distancia_voo_medio_regiao")
).toPandas()


,region_D,distancia_voo_medio_regiao
0,ALASKA,1742
1,MAINLAND-EAST,2043
2,MAINLAND-WEST,868


Pergunta 15

In [122]:
df.select("distance", "origin", "dest").groupBy("origin", "dest").agg(
    F.ceil(F.avg("distance")).alias("distancia_voo_medio_rota")
).toPandas()


,origin,dest,distancia_voo_medio_rota
0,SEA,RNO,564
1,SEA,DTW,1927
2,SEA,CLE,2021
3,SEA,LAX,954
4,PDX,SEA,129
...,...,...,...
108,SEA,JFK,2422
109,SEA,FAI,1533
110,PDX,HOU,1843
111,PDX,BUR,817


Pergunta 16

In [123]:
df.select(F.year("dep_datetime").alias("ano"), "distance").groupBy("ano").agg(
    F.ceil(F.sum("distance")).alias("distancia_voo_acumulado_ano")
).toPandas()


,ano,distancia_voo_acumulado_ano
0,2013,106
1,2014,12081410


Pergunta 17

In [124]:
df.select(F.year("dep_datetime").alias("ano"), "distance", "region_D").groupBy(
    "ano", "region_D"
).agg(
    F.ceil(F.sum(F.col("distance"))).alias("distancia_acumulado_regiao")
).toPandas()


,ano,region_D,distancia_acumulado_regiao
0,2014,ALASKA,1762553
1,2014,MAINLAND-EAST,4378902
2,2014,MAINLAND-WEST,5939955
3,2013,MAINLAND-WEST,106


Pergunta 18

In [128]:
# df.select("seats", "origin","dest").groupBy("origin", "dest").agg(F.round(F.avg("seats"), 3).alias("passageiros_medio_rota")).toPandas()


Pergunta 19

In [131]:
# df.select(F.year("dep_datetime").alias("ano"),"seats").groupBy("ano").agg(F.sum("seats").alias("passageiros_acumulado_ano")).toPandas()


Pergunta 20

In [133]:
df.select("dest").groupBy("dest").count().orderBy(
    "count", ascending=False
).limit(1).toPandas()


,dest,count
0,SFO,787


Pergunta 21

In [136]:
# df.select("dest", "seats").groupBy("dest").agg(F.sum("seats").alias("destino_mais_passageiros")).orderBy("destino_mais_passageiros", ascending= False).limit(1).toPandas()


Pergunta 22

In [137]:
df.select("origin", "dest", "distance").where(F.col("origin") == "PDX").orderBy(
    "distance", ascending=False
).limit(1).toPandas()


,origin,dest,distance
0,PDX,LIH,2631


Pergunta 23

In [141]:
df.groupBy("dest", F.month("dep_datetime").alias("mes")).count().orderBy(
    "count", ascending=False
).groupBy("dest").mean().orderBy("avg(count)", ascending=False).limit(
    1
).toPandas()


,dest,avg(mes),avg(count)
0,SFO,6.5,65.583333


Pergunta 24

In [143]:
# df.groupBy("model").count().orderBy('count', ascending= False).limit(1).toPandas()


Pergunta 25

In [145]:
# df.where(F.col("model").isNotNull()).groupBy("model", "dest").count().orderBy('count', ascending= False).limit(1).toPandas()


Pergunta 26

In [147]:
# df.groupBy("haul_duration").agg(F.avg("engines").alias("media_motores_categoria")).toPandas()


Pergunta 27

In [148]:
df.groupBy("dep_season").count().orderBy("count", ascending=False).limit(
    1
).toPandas()


,dep_season,count
0,SUMMER,2918


Pergunta 28

In [149]:
df.groupBy("dep_season", "dest").count().orderBy(
    "count", ascending=False
).limit(1).toPandas()


,dep_season,dest,count
0,SUMMER,SFO,217


Pergunta 29

In [150]:
df.where(
    (F.col("dep_delay_category") != "ANTECIPATED")
    & (F.col("dep_delay_category") != "INTIME")
).groupBy("dep_delay_category").count().orderBy("count", ascending=False).limit(
    1
).toPandas()


,dep_delay_category,count
0,MINOR,3065


Pergunta 30

In [151]:
df.where(
    (F.col("dep_delay_category") != "ANTECIPATED")
    & (F.col("dep_delay_category") != "INTIME")
).groupBy("dep_delay_category", "origin", "dest").count().orderBy(
    "count", ascending=False
).limit(
    1
).toPandas()


,dep_delay_category,origin,dest,count
0,MINOR,SEA,SFO,159
